In [1]:
!pip install langchain-google-genai
!pip install faiss-cpu
!pip install python-dotenv
!pip install pandas
!pip install numpy
!pip install requests
!pip install beautifulsoup4
!pip install langchain
!pip install langchain-community

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
# Update this import
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, WebBaseLoader
import pandas as pd
import numpy as np
from typing import List, Dict, Optional
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime, timedelta
import re

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# Cell 1: Install required packages
!pip install python-dotenv PyPDF2 python-docx pandas numpy

In [4]:
# Import necessary libraries
import PyPDF2
import docx
from pathlib import Path

# Load environment variables from .env file
load_dotenv()

# Configuration class to store all settings
class Config:
    def __init__(self):
        # Load configuration from environment variables
        self.chunk_size = int(os.getenv('CHUNK_SIZE', 1000))
        self.chunk_overlap = int(os.getenv('CHUNK_OVERLAP', 200))
        self.debug_mode = os.getenv('DEBUG_MODE', 'False').lower() == 'true'
        
    def __str__(self):
        return f"Configuration:\nChunk Size: {self.chunk_size}\nChunk Overlap: {self.chunk_overlap}\nDebug Mode: {self.debug_mode}"

# Initialize configuration
config = Config()

# Print configuration for verification
if config.debug_mode:
    print(config)

Configuration:
Chunk Size: 1000
Chunk Overlap: 200
Debug Mode: True


In [5]:
class DocumentProcessor:
    def __init__(self, config: Config):
        self.config = config
        self.supported_formats = {
            '.pdf': self._process_pdf,
            '.docx': self._process_docx,
            '.txt': self._process_txt
        }
    
    def process_document(self, file_path: str) -> List[str]:
        """
        Process a document and return its content in chunks.
        
        Args:
            file_path (str): Path to the document
            
        Returns:
            List[str]: List of text chunks
        """
        file_path = Path(file_path)
        if not file_path.exists():
            raise FileNotFoundError(f"File not found: {file_path}")
            
        file_extension = file_path.suffix.lower()
        if file_extension not in self.supported_formats:
            raise ValueError(f"Unsupported file format: {file_extension}")
            
        # Process the document based on its format
        content = self.supported_formats[file_extension](file_path)
        
        # Split content into chunks
        chunks = self._create_chunks(content)
        
        if self.config.debug_mode:
            print(f"Processed {file_path.name}")
            print(f"Created {len(chunks)} chunks")
            
        return chunks
    
    def _process_pdf(self, file_path: Path) -> str:
        """Extract text from PDF file."""
        text = ""
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text
    
    def _process_docx(self, file_path: Path) -> str:
        """Extract text from Word document."""
        doc = docx.Document(file_path)
        return "\n".join([paragraph.text for paragraph in doc.paragraphs])
    
    def _process_txt(self, file_path: Path) -> str:
        """Extract text from plain text file."""
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    
    def _create_chunks(self, text: str) -> List[str]:
        """
        Split text into overlapping chunks.
        
        Args:
            text (str): Input text to split
            
        Returns:
            List[str]: List of text chunks
        """
        chunks = []
        start = 0
        text_length = len(text)
        
        while start < text_length:
            # Calculate end position for current chunk
            end = start + self.config.chunk_size
            
            # If this is not the first chunk, include some overlap
            if start > 0:
                start = start - self.config.chunk_overlap
            
            # Get the chunk
            chunk = text[start:end]
            chunks.append(chunk)
            
            # Move to next chunk
            start = end
            
        return chunks

In [6]:
# Cell: Process all books in the study materials directory
def process_all_books():
    """Process all books in the study materials directory."""
    processor = DocumentProcessor(config)
    all_chunks = []
    
    # Get all PDF files in directory
    directory = Path("Study_Materials")
    for file_path in directory.glob('*.pdf'):
        try:
            print(f"\nProcessing {file_path.name}...")
            chunks = processor.process_document(str(file_path))
            all_chunks.extend(chunks)
            print(f"Successfully processed {file_path.name} into {len(chunks)} chunks")
        except Exception as e:
            print(f"Error processing {file_path.name}: {str(e)}")
    
    print(f"\nTotal chunks processed from all books: {len(all_chunks)}")
    return all_chunks

# Run the processing
all_chunks = process_all_books()


Processing Atomic Habits by James Clear.pdf.pdf...
Processed Atomic Habits by James Clear.pdf.pdf
Created 464 chunks
Successfully processed Atomic Habits by James Clear.pdf.pdf into 464 chunks

Processing Deep_Work_Rules_for_focused_success_in_a.pdf...
Processed Deep_Work_Rules_for_focused_success_in_a.pdf
Created 451 chunks
Successfully processed Deep_Work_Rules_for_focused_success_in_a.pdf into 451 chunks

Processing Getting Things Done by David Allen.pdf...
Processed Getting Things Done by David Allen.pdf
Created 659 chunks
Successfully processed Getting Things Done by David Allen.pdf into 659 chunks

Total chunks processed from all books: 1574


In [7]:
# Cell 2: Import necessary libraries and set up FAISS with TF-IDF
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import List, Dict, Any
import os
from dotenv import load_dotenv
import pickle

# Load environment variables
load_dotenv()

class VectorDatabase:
    def __init__(self, config: Config):
        self.config = config
        # Initialize TF-IDF vectorizer
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            stop_words='english',
            ngram_range=(1, 2)  # Use both unigrams and bigrams
        )
        
        # Initialize FAISS index
        self.dimension = 1000  # Dimension for TF-IDF vectors
        self.index = None  # Will be initialized when we add documents
        
        # Store documents and metadata
        self.documents = []
        self.metadata = []
        
        # Create directory for persistence
        os.makedirs("vector_db", exist_ok=True)
    
    def add_documents(self, chunks: List[str], metadata: List[Dict[str, Any]] = None):
        """
        Add document chunks to the FAISS index.
        
        Args:
            chunks (List[str]): List of text chunks
            metadata (List[Dict[str, Any]], optional): List of metadata for each chunk
        """
        if metadata is None:
            metadata = [{"source": "unknown"} for _ in chunks]
            
        # Generate TF-IDF embeddings
        embeddings = self.vectorizer.fit_transform(chunks).toarray()
        
        # Initialize FAISS index if not already done
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        
        # Add to FAISS index
        self.index.add(embeddings.astype('float32'))
        
        # Store documents and metadata
        self.documents.extend(chunks)
        self.metadata.extend(metadata)
        
        if self.config.debug_mode:
            print(f"Added {len(chunks)} chunks to FAISS index")
            print(f"Vector dimension: {embeddings.shape[1]}")
    
    def search(self, query: str, n_results: int = 5) -> Dict[str, Any]:
        """
        Search for relevant chunks in the FAISS index.
        
        Args:
            query (str): Search query
            n_results (int): Number of results to return
            
        Returns:
            Dict[str, Any]: Dictionary containing relevant chunks and metadata
        """
        if not self.documents:
            return {'documents': [], 'metadatas': [], 'distances': []}
            
        # Generate query embedding
        query_embedding = self.vectorizer.transform([query]).toarray()
        
        # Search in FAISS index
        distances, indices = self.index.search(
            query_embedding.astype('float32'), 
            n_results
        )
        
        # Prepare results
        results = {
            'documents': [self.documents[idx] for idx in indices[0]],
            'metadatas': [self.metadata[idx] for idx in indices[0]],
            'distances': distances[0].tolist()
        }
        
        return results
    
    def save(self, path: str = "vector_db"):
        """Save the FAISS index and associated data."""
        if self.index is not None:
            # Save FAISS index
            faiss.write_index(self.index, os.path.join(path, "index.faiss"))
            
            # Save vectorizer
            with open(os.path.join(path, "vectorizer.pkl"), "wb") as f:
                pickle.dump(self.vectorizer, f)
            
            # Save documents and metadata
            with open(os.path.join(path, "documents.pkl"), "wb") as f:
                pickle.dump(self.documents, f)
            with open(os.path.join(path, "metadata.pkl"), "wb") as f:
                pickle.dump(self.metadata, f)
    
    def load(self, path: str = "vector_db"):
        """Load the FAISS index and associated data."""
        # Load FAISS index
        self.index = faiss.read_index(os.path.join(path, "index.faiss"))
        
        # Load vectorizer
        with open(os.path.join(path, "vectorizer.pkl"), "rb") as f:
            self.vectorizer = pickle.load(f)
        
        # Load documents and metadata
        with open(os.path.join(path, "documents.pkl"), "rb") as f:
            self.documents = pickle.load(f)
        with open(os.path.join(path, "metadata.pkl"), "rb") as f:
            self.metadata = pickle.load(f)

In [8]:
# Cell 3: Test the vector database
def test_vector_database():
    """Test the vector database with sample chunks."""
    # Initialize vector database
    vector_db = VectorDatabase(config)
    
    # Sample chunks for testing
    sample_chunks = [
        "Time management is crucial for productivity. Plan your day in advance.",
        "Break down large tasks into smaller, manageable pieces.",
        "Use the Pomodoro technique: work for 25 minutes, then take a 5-minute break.",
        "Prioritize your tasks using the Eisenhower Matrix.",
        "Regular exercise and proper sleep improve productivity."
    ]
    
    # Add chunks to database
    metadata = [{"source": "productivity_tips"} for _ in sample_chunks]
    vector_db.add_documents(sample_chunks, metadata)
    
    # Test search functionality
    test_queries = [
        "How to manage time effectively?",
        "What's the best way to stay productive?",
        "How to break down tasks?"
    ]
    
    print("Testing search functionality:")
    for query in test_queries:
        print(f"\nQuery: {query}")
        results = vector_db.search(query, n_results=2)
        print("\nTop 2 relevant chunks:")
        for i, (doc, metadata, distance) in enumerate(zip(
            results['documents'], 
            results['metadatas'], 
            results['distances']
        ), 1):
            print(f"\n{i}. {doc}")
            print(f"Source: {metadata['source']}")
            print(f"Distance: {distance:.4f}")

# Run the test
test_vector_database()

Added 5 chunks to FAISS index
Vector dimension: 56
Testing search functionality:

Query: How to manage time effectively?

Top 2 relevant chunks:

1. Time management is crucial for productivity. Plan your day in advance.
Source: productivity_tips
Distance: 1.4377

2. Use the Pomodoro technique: work for 25 minutes, then take a 5-minute break.
Source: productivity_tips
Distance: 2.0000

Query: What's the best way to stay productive?

Top 2 relevant chunks:

1. Break down large tasks into smaller, manageable pieces.
Source: productivity_tips
Distance: 1.0000

2. Prioritize your tasks using the Eisenhower Matrix.
Source: productivity_tips
Distance: 1.0000

Query: How to break down tasks?

Top 2 relevant chunks:

1. Break down large tasks into smaller, manageable pieces.
Source: productivity_tips
Distance: 1.2890

2. Prioritize your tasks using the Eisenhower Matrix.
Source: productivity_tips
Distance: 1.6121


In [9]:
# Cell 2: Set up Google Gemini integration
import google.generativeai as genai
from typing import Dict, Any, List
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class LLMIntegration:
    def __init__(self, config: Config):
        self.config = config
        # Configure Google Gemini
        genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
        
        # Initialize the model
        self.model = genai.GenerativeModel('models/gemini-2.0-flash-exp')
        
        # Set up the chat
        self.chat = self.model.start_chat(history=[])
        
        if self.config.debug_mode:
            print("LLM Integration initialized successfully")
    
    def generate_response(self, prompt: str, context: List[str] = None) -> str:
        """
        Generate a response using the LLM.
        
        Args:
            prompt (str): The input prompt
            context (List[str], optional): Additional context for the response
            
        Returns:
            str: Generated response
        """
        try:
            # Combine context and prompt if context is provided
            full_prompt = prompt
            if context:
                context_text = "\n".join(context)
                full_prompt = f"Context:\n{context_text}\n\nQuestion: {prompt}"
            
            # Generate response
            response = self.chat.send_message(full_prompt)
            
            return response.text
            
        except Exception as e:
            if self.config.debug_mode:
                print(f"Error generating response: {str(e)}")
            return f"Error: {str(e)}"
    
    def reset_chat(self):
        """Reset the chat history."""
        self.chat = self.model.start_chat(history=[])

In [10]:
# Cell 3: Combine Vector Database with LLM to create RAG system
class RAGSystem:
    def __init__(self, config: Config):
        self.config = config
        self.vector_db = VectorDatabase(config)
        self.llm = LLMIntegration(config)
    
    def query(self, question: str, n_results: int = 3) -> str:
        """
        Query the RAG system with a question.
        
        Args:
            question (str): The question to ask
            n_results (int): Number of relevant chunks to retrieve
            
        Returns:
            str: Generated response
        """
        # Search for relevant chunks
        results = self.vector_db.search(question, n_results=n_results)
        
        # Extract relevant context
        context = results['documents']
        
        # Generate response using LLM
        response = self.llm.generate_response(question, context)
        
        return response

In [11]:
# Cell: Store processed chunks in vector database
def store_processed_chunks(all_chunks):
    """Store the processed chunks in the vector database."""
    # Initialize RAG system
    rag = RAGSystem(config)
    
    # Create metadata for chunks
    # You can modify this based on your needs
    metadata = [{"source": "productivity_book"} for _ in all_chunks]
    
    # Add chunks to vector database
    rag.vector_db.add_documents(all_chunks, metadata)
    
    # Save the vector database
    rag.vector_db.save()
    
    print(f"Successfully stored {len(all_chunks)} chunks in vector database")
    
    # Test the stored knowledge
    test_queries = [
        "What are the key productivity techniques?",
        "How can I improve my time management?",
        "What's the best way to stay focused?"
    ]
    
    print("\nTesting stored knowledge:")
    for query in test_queries:
        print(f"\nQuery: {query}")
        response = rag.query(query)
        print(f"Response: {response}")
        
        # Add a small delay to avoid rate limiting
        import time
        time.sleep(1)
    
    return rag

# Store the chunks we processed earlier
rag_system = store_processed_chunks(all_chunks)

LLM Integration initialized successfully
Added 1574 chunks to FAISS index
Vector dimension: 1000
Successfully stored 1574 chunks in vector database

Testing stored knowledge:

Query: What are the key productivity techniques?
Response: Based on the text provided, the key productivity techniques are:

*   **Capturing everything:** Getting all the things that need to be done or have usefulness out of your head and into a trusted system. This includes tasks, ideas, and information.
*   **Making front-end decisions:** Processing all the inputs that come into your life to create a clear inventory of actionable "next actions" that can be implemented or renegotiated.
*   **Curating and coordinating:** Managing all of your commitments at different levels to stay organized and focused.
*   **Mastering the basics**: Building proficiency with the fundamentals, which will help to feel less buried in daily tasks.
*   **Clarifying and organizing**: Changing the way you clarify and organize all the th

In [14]:
def interactive_flex_chat(rag_system):
    print("👋 Hello! I'm Flex AI, your productivity coach.")
    print("Let's work together to create your custom productivity plan.")
    print("You can ask for a new plan, advice, or just chat. (Type 'exit', 'bye', or 'quit' to leave.)\n")
    
    # Gather user profile once per session
    questions = [
        ("goal", "What is your main productivity goal or project?"),
        ("duration", "What is your target completion date (or how many days/weeks do you want to work on this?)"),
        ("wake_time", "What time do you usually wake up?"),
        ("sleep_time", "What time do you usually go to sleep?"),
        ("focus_hours", "When do you feel most focused during the day?"),
        ("daily_hours", "How many hours per day can you dedicate to this goal?"),
        ("work_style", "Do you prefer working in long sessions or short bursts?"),
        ("habits", "Any specific habits or routines you want to include?"),
        ("rest_days", "Any days you want to take off or rest?")
    ]
    user_info = {}
    for key, q in questions:
        answer = input(f"Flex AI: {q}\nYou: ")
        if answer.strip().lower() in ['exit', 'bye', 'quit']:
            print("Flex AI: No problem! Come back anytime you want to plan your productivity journey. 🚀")
            return
        user_info[key] = answer

    print("\nFlex AI: Thank you! You can now ask for a plan, advice, or anything else related to productivity.\n")
    
    while True:
        user_input = input("You: ")
        if user_input.strip().lower() in ['exit', 'bye', 'quit']:
            print("Flex AI: Goodbye! Stay productive! 🚀")
            break

        # If the user asks for a plan, generate a new plan using their profile and RAG
        if any(word in user_input.lower() for word in ['plan', 'schedule', 'roadmap']):
            rag_context = rag_system.vector_db.search(user_info['goal'], n_results=5)['documents']
            context_text = "\n".join(rag_context)
            plan_prompt = (
                f"Context from productivity literature:\n{context_text}\n\n"
                f"Create a detailed, structured productivity plan for the following user profile:\n"
                f"Goal: {user_info['goal']}\n"
                f"Target duration/completion: {user_info['duration']}\n"
                f"Wake time: {user_info['wake_time']}\n"
                f"Sleep time: {user_info['sleep_time']}\n"
                f"Focus hours: {user_info['focus_hours']}\n"
                f"Daily work hours: {user_info['daily_hours']}\n"
                f"Work style: {user_info['work_style']}\n"
                f"Habits/routines: {user_info['habits']}\n"
                f"Rest days: {user_info['rest_days']}\n\n"
                f"Instructions:\n"
                f"- Break the main goal into milestones and small actionable tasks.\n"
                f"- Distribute tasks over the available days and focus hours.\n"
                f"- Suggest a daily schedule and routines.\n"
                f"- Present the plan in a clear, structured format (with sections for goals, milestones, tasks, and schedule).\n"
                f"- Make the plan motivating and easy to follow.\n"
            )
            response = rag_system.llm.generate_response(plan_prompt)
            print(f"\nFlex AI (Your Plan):\n{response}\n")
        else:
            # For any other input, use the RAG system to answer or give advice
            rag_context = rag_system.vector_db.search(user_input, n_results=3)['documents']
            context_text = "\n".join(rag_context)
            advice_prompt = (
                f"Context from productivity literature:\n{context_text}\n\n"
                f"User question: {user_input}\n"
                f"Based on the above context and best productivity practices, provide a helpful, actionable answer."
            )
            response = rag_system.llm.generate_response(advice_prompt)
            print(f"\nFlex AI:\n{response}\n")

# Usage:
interactive_flex_chat(rag_system)

👋 Hello! I'm Flex AI, your productivity coach.
Let's work together to create your custom productivity plan.
You can ask for a new plan, advice, or just chat. (Type 'exit', 'bye', or 'quit' to leave.)

Flex AI: No problem! Come back anytime you want to plan your productivity journey. 🚀


In [16]:
def form_mode_simple(rag_system):
    print("📝 Flex AI Form Mode: Quick Productivity Plan")
    print("Please enter your preferences to generate a personalized plan.\n")

    # Collect user input
    user_info = {}
    user_info['wake_time'] = input("Wake Up Time (e.g., 07:00 AM): ")
    user_info['sleep_time'] = input("Sleep Time (e.g., 11:00 PM): ")
    user_info['focus_periods'] = input("Focus Periods Per Day (e.g., 4): ")
    user_info['break_duration'] = input("Break Duration (minutes, e.g., 15): ")
    user_info['goal'] = input("Primary Goal (e.g., Complete React Project): ")

    # Generate plan using RAG system
    print("\nGenerating your personalized productivity plan...\n")
    rag_context = rag_system.vector_db.search(user_info['goal'], n_results=5)['documents']
    context_text = "\n".join(rag_context)

    plan_prompt = (
        f"Context from productivity literature:\n{context_text}\n\n"
        f"Create a detailed, actionable productivity plan for the following user:\n"
        f"- Goal: {user_info['goal']}\n"
        f"- Wake Up Time: {user_info['wake_time']}\n"
        f"- Sleep Time: {user_info['sleep_time']}\n"
        f"- Focus Periods Per Day: {user_info['focus_periods']}\n"
        f"- Break Duration: {user_info['break_duration']} minutes\n\n"
        f"Instructions:\n"
        f"- Break the main goal into milestones and actionable tasks.\n"
        f"- Distribute tasks over the available focus periods.\n"
        f"- Suggest a daily schedule including breaks.\n"
        f"- Present the plan in a clear, structured format.\n"
        f"- Make the plan motivating and easy to follow.\n"
    )

    response = rag_system.llm.generate_response(plan_prompt)
    print("\n=== Your Personalized Productivity Plan ===\n")
    print(response)

    # Optionally, save the plan
    save_response = input("\nWould you like to save this plan? (yes/no): ")
    if save_response.lower() in ['yes', 'y']:
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"productivity_plan_{timestamp}.txt"
        with open(filename, "w") as f:
            f.write(f"=== Productivity Plan ===\n\n")
            f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            for key, value in user_info.items():
                f.write(f"{key}: {value}\n")
            f.write(f"\nPlan:\n{response}")
        print(f"\nPlan saved to {filename}")

# Usage:
form_mode_simple(rag_system)

📝 Flex AI Form Mode: Quick Productivity Plan
Please enter your preferences to generate a personalized plan.


Generating your personalized productivity plan...


=== Your Personalized Productivity Plan ===

Okay, here's a detailed, actionable productivity plan to help you master Python, designed to be motivating and easy to follow.

**User Profile:**

*   Goal: Learn Python
*   Wake Up: 7:00 AM
*   Sleep: 12:00 AM (Midnight)
*   Focus Periods: 3 per day
*   Break Duration: 25 minutes

**Overall Strategy:**

This plan uses a combination of structured learning, hands-on practice, and project-based learning. It emphasizes consistent, focused effort with regular breaks to prevent burnout.  The plan assumes a beginner level. If you have some prior experience, adjust the milestones accordingly.

**Phase 1: Core Concepts (Weeks 1-4)**

*   **Milestone 1: Python Fundamentals** (Complete by end of Week 2)
*   **Milestone 2: Data Structures & Control Flow** (Complete by end of Week 4)

**Phase 2

In [18]:
import requests

def web_search_duckduckgo(query):
    url = "https://api.duckduckgo.com/"
    params = {"q": query, "format": "json", "no_redirect": 1, "no_html": 1}
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    # Try to get the abstract or related topics
    if data.get("AbstractText"):
        return [data["AbstractText"]]
    elif data.get("RelatedTopics"):
        return [topic.get("Text", "") for topic in data["RelatedTopics"] if topic.get("Text")]
    else:
        return ["No instant answer found."]
        
# Example usage:
print(web_search_duckduckgo("latest productivity techniques 2024"))

['No instant answer found.']


In [19]:
def generate_roadmap(rag_system, user_goal):
    print("🚀 Flex AI Roadmap Generator")
    print(f"Generating a roadmap for: {user_goal}\n")

    # Get relevant context from RAG
    rag_context = rag_system.vector_db.search(user_goal, n_results=5)['documents']
    context_text = "\n".join(rag_context)

    # Prompt for roadmap generation
    roadmap_prompt = (
        f"Context from productivity literature:\n{context_text}\n\n"
        f"Create a detailed, actionable roadmap for the following goal:\n"
        f"Goal: {user_goal}\n\n"
        f"Instructions:\n"
        f"- Break the goal into 3-6 major milestones.\n"
        f"- For each milestone, list 3-5 actionable tasks.\n"
        f"- If relevant, show dependencies (e.g., 'Task B depends on Task A').\n"
        f"- Suggest a logical order or timeline for milestones.\n"
        f"- Present the roadmap in a clear, structured format (use markdown-style headings and lists).\n"
        f"- Make the roadmap motivating and easy to follow.\n"
    )

    roadmap = rag_system.llm.generate_response(roadmap_prompt)
    print("\n=== Your Personalized Roadmap ===\n")
    print(roadmap)

    # Optionally, save the roadmap
    save_response = input("\nWould you like to save this roadmap? (yes/no): ")
    if save_response.lower() in ['yes', 'y']:
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"roadmap_{timestamp}.md"
        with open(filename, "w") as f:
            f.write(f"# Roadmap for: {user_goal}\n\n")
            f.write(roadmap)
        print(f"\nRoadmap saved to {filename}")

# Usage example:
user_goal = "Complete React Project"
generate_roadmap(rag_system, user_goal)

🚀 Flex AI Roadmap Generator
Generating a roadmap for: Complete React Project


=== Your Personalized Roadmap ===

Okay, here's a detailed, actionable roadmap for completing a React project, designed for clarity, structure, and motivation. This assumes you have some basic understanding of HTML, CSS, and JavaScript. If not, add a "Learn the Basics" milestone first!

**Overall Goal:** Complete a React Project

**Project Selection (Important Pre-Step):**

Before diving in, choose a project that is:

*   **Interesting to you:** This will keep you motivated.
*   **Challenging, but achievable:** Avoid projects that are too complex for your current skill level, but also don't pick something too easy.
*   **Well-defined:** Know what the end product should look like.

For example, a good beginner React project could be a to-do list application, a simple calculator, a personal portfolio website, or a basic e-commerce product display.

**Roadmap Structure:**

This roadmap is divided into 5 major m